# Coursera IBM Data Science Capstone Exercise
## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

#### Edward Walthew

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 102kB 20.3MB/s ta 0:00:01
Libraries imported.


### First submission: Scraping Table for Wikipedia and Creating the DataFrame

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Read table in, contained in the first dataframe
res = pd.read_html(url)[0]

#Rows where neighborhood is 'Not assigned' but Borough is then the neighborhood is the same as the borough
res['Neighborhood'] = res['Neighborhood'].mask(res['Neighborhood']=='Not assigned',res['Borough'], axis=0)
res['Neighborhood'] = res['Neighborhood'].mask(res['Neighborhood'].isna(),res['Borough'], axis=0)

#Remove rows where Borough is 'Not assigned'
res = res.mask(res['Borough']=='Not assigned').dropna(axis=0)

#Group by postal code
res = res.groupby('Postal Code').apply(lambda x: x).reset_index(drop=True)
display(res.head())

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [3]:
res.shape

(103, 3)

### Second submission: Adding location data to DataFrame

In [4]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [5]:
!pip install geocoder

     |████████████████████████████████| 102kB 7.7MB/s ta 0:00:011


In [6]:
import geocoder # import geocoder
#read from CVS stored in IBM Cloud Object Storage in above hidden cell
backup = df_data_0.set_index('Postal Code')

def get_coords(postal_code):
    # initialize your variable to None
    lat_lng_coords = None

    i = 0
    # loop until you get the coordinates #request never seems to succeed
#     while(lat_lng_coords is None):
#         g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#         lat_lng_coords = g.latlng
#         if(i == 10):
#             print('Failed to retrieve coordinates for ' + postal_code)
#             break
#         i+=1

    if(lat_lng_coords is None):
        latitude = backup.loc[postal_code,'Latitude']
        longitude = backup.loc[postal_code,'Longitude']
    else:
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
    
    return [latitude, longitude]

In [7]:
res[['Latitude','Longitude']] = res['Postal Code'].apply(lambda x: pd.Series(get_coords(x)))
display(res.head())
display(res.shape)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


(103, 5)

### Third Submission: Exploration, Clustering and Analysis

In [8]:
# The code was removed by Watson Studio for sharing.

In [9]:
VERSION = '20180605' # Foursquare API version

In [10]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Map shows map of Toronto with Postal Code coordinates marked.
##### Those Postal Codes located in Boroughs with 'Toronto' in the name are marked red, others are marked blue.

In [11]:
import re
patt = re.compile('Toronto')
toronto_data = res
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
count_toronto = 0
count_other = 0
# add markers to map
for lat, lng, label1, label2, label3 in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Postal Code'], toronto_data['Neighborhood'], toronto_data['Borough']):
    label = "{} ({}:{})".format(label1, label3, label2)
    label = folium.Popup(label, parse_html=True)
    if patt.search(label3) is not None: 
        color = 'red'
        count_toronto+=1
    else: 
        color = 'blue'
        count_other+=1
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=color,
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
print('Postal Codes in "Toronto" Boroughs: {}'.format(count_toronto))
print('Postal Codes in other Boroughs: {}'.format(count_other)) 
map_toronto

Postal Codes in "Toronto" Boroughs: 39
Postal Codes in other Boroughs: 64


#### Not limiting the Borough selection, collecting venue data

In [96]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 
                  'Postal Code Latitude', 
                  'Postal Code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [97]:
toronto_venues = getNearbyVenues(names=toronto_data['Postal Code'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )


M3A
M4A
M5A
M6A
M7A
M9A
M1B
M3B
M4B
M5B
M6B
M9B
M1C
M3C
M4C
M5C
M6C
M9C
M1E
M4E
M5E
M6E
M1G
M4G
M5G
M6G
M1H
M2H
M3H
M4H
M5H
M6H
M1J
M2J
M3J
M4J
M5J
M6J
M1K
M2K
M3K
M4K
M5K
M6K
M1L
M2L
M3L
M4L
M5L
M6L
M9L
M1M
M2M
M3M
M4M
M5M
M6M
M9M
M1N
M2N
M3N
M4N
M5N
M6N
M9N
M1P
M2P
M4P
M5P
M6P
M9P
M1R
M2R
M4R
M5R
M6R
M7R
M9R
M1S
M4S
M5S
M6S
M1T
M4T
M5T
M1V
M4V
M5V
M8V
M9V
M1W
M4W
M5W
M8W
M9W
M1X
M4X
M5X
M8X
M4Y
M7Y
M8Y
M8Z


In [98]:
display(toronto_venues.shape)
toronto_venues.head()

(2127, 7)

,Postal Code,Postal Code Latitude,Postal Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,43.753259,-79.329656,TTC stop #8380,43.752672,-79.326351,Bus Stop
2,M3A,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,M4A,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,M4A,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [99]:
toronto_venues['Postal Code'].nunique()

100

In [100]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 272 uniques categories.


#### Using Onehot encoding to prepare data for clustering

In [126]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Postal Code'] = toronto_venues['Postal Code'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Postal Code,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M3A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [127]:
toronto_onehot.shape

(2127, 273)

#### Grouping by Postal Code showing frequency (as a sum rather than mean) of venue categories.
##### The first (toronto_grouped) is the sum normalised by the maximum in the column. This is used for clustering as very common (over the entire city not just the postal code) venue types will be weighted less per instance to even their influence on distance metrics.
##### The second (toronto_grouped_sum) is the unnormalised sum.

In [264]:
#toronto_grouped = toronto_onehot.groupby('Postal Code').sum().div(toronto_onehot.drop(columns='Postal Code').sum(axis=0)).reset_index()
toronto_grouped = toronto_onehot.groupby('Postal Code').sum().div(toronto_onehot.drop(columns='Postal Code').sum().max(axis=0)).reset_index()
#toronto_grouped = toronto_onehot.groupby('Postal Code').mean().reset_index()
toronto_grouped_sum = toronto_onehot.groupby('Postal Code').sum().reset_index()
toronto_grouped

,Postal Code,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00000,

In [265]:
toronto_grouped.shape

(100, 273)

#### Replacing missing postal codes as two postal codes are missing having no venues returned

In [266]:
omitted_pc = list(set(toronto_data['Postal Code'])-set(toronto_grouped['Postal Code']))
omitted_pc

['M1X', 'M9B', 'M2M']

In [267]:
omitted_df = pd.DataFrame({'Postal Code':omitted_pc}, columns=toronto_grouped.columns).fillna(value=0)
toronto_grouped = toronto_grouped.append(omitted_df).reset_index(drop=True)
toronto_grouped_sum = toronto_grouped_sum.append(omitted_df).reset_index(drop=True)
toronto_grouped.head()

,Postal Code,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.0,

#### Printing top five most frequent venues in each Postal Code (using the unnormalized sum)

In [269]:
num_top_venues = 5
def printTop5(toronto_grouped):
    for hood in toronto_grouped['Postal Code']:
        print("----"+hood+"----")
        temp = toronto_grouped[toronto_grouped['Postal Code'] == hood].T.reset_index()
        temp.columns = ['venue','freq']
        temp = temp.iloc[1:]
        temp['freq'] = temp['freq'].astype(float)
        temp = temp.round({'freq': 2})
        print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
        print('\n')

printTop5(toronto_grouped_sum)

----M1B----
                             venue  freq
0             Fast Food Restaurant   1.0
1               Mexican Restaurant   0.0
2              Monument / Landmark   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


----M1C----
                             venue  freq
0                              Bar   1.0
1                Accessories Store   0.0
2        Middle Eastern Restaurant   0.0
3              Monument / Landmark   0.0
4  Molecular Gastronomy Restaurant   0.0


----M1E----
               venue  freq
0       Intersection   1.0
1     Medical Center   1.0
2     Breakfast Spot   1.0
3      Moving Target   1.0
4  Electronics Store   1.0


----M1G----
                       venue  freq
0                Coffee Shop   2.0
1         Mexican Restaurant   1.0
2          Convenience Store   1.0
3          Korean Restaurant   1.0
4  Middle Eastern Restaurant   0.0


----M1H----
                 venue  freq
0               Lounge   1.0
1   Athlet

#### Display 10 most common venues for each postcode. 
##### However some of the postcodes have less than 10 venues present.
##### Common Venue ranking is based on the total unnormalized sum.
##### Distinctive Venue ranking is based on the normalized sum, because more common (i.e. less distinctive) venues will have less weight per instance.

In [270]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    ind = 0
    for i in range(0,num_top_venues):
        if row_categories_sorted[i] > 0:
            ind = i+1
        else:
            break 
    
    top_venues = np.copy(row_categories_sorted.index.values[0:num_top_venues]);
    
    if ind != num_top_venues:
        top_venues[ind:num_top_venues] = 'N/A'
    
    return top_venues

In [285]:
def getTop10(toronto_grouped, metric_label):
    num_top_venues = 10

    indicators = ['st', 'nd', 'rd']

    # create columns according to number of top venues
    columns = ['Postal Code']
    for ind in np.arange(num_top_venues):
        try:
            columns.append('{}{} Most {} Venue'.format(ind+1, indicators[ind], metric_label))
        except:
            columns.append('{}th Most {} Venue'.format(ind+1, metric_label))

    # create a new dataframe
    neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
    neighborhoods_venues_sorted['Postal Code'] = toronto_grouped['Postal Code']

    for ind in np.arange(toronto_grouped.shape[0]):
        neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

    return neighborhoods_venues_sorted

neighborhoods_venues_sorted = getTop10(toronto_grouped_sum, 'Common')
display(neighborhoods_venues_sorted)

neighborhoods_venues_sorted_distinct = getTop10(toronto_grouped, 'Distinctive')
display(neighborhoods_venues_sorted_distinct)

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1,M1C,Bar,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
2,M1E,Electronics Store,Moving Target,Rental Car Location,Mexican Restaurant,Breakfast Spot,Bank,Intersection,Medical Center,N/A,N/A
3,M1G,Coffee Shop,Mexican Restaurant,Korean Restaurant,Convenience Store,N/A,N/A,N/A,N/A,N/A,N/A
4,M1H,Hakka Restaurant,Fried Chicken Joint,Lounge,Bank,Athletics & Sports,Caribbean Restaurant,Thai Restaurant,Gas Station,Bakery,N/A
5,M1J,Pizza Place,Playground,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
6,M1K,Chinese Restaurant,Bus Station,Coffee Shop,Hobby Shop,Department Store,N/A,N/A,N/A,N/A,N/A
7,M1L,Bakery,Park,Ice Cream Shop,Soccer Field,Bus Station,Bus Line,N/A,N/A,N/A,N/A
8,M1M,American Restaurant,Motel,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
9,M1N,College Stadium,Skating Rink,Café,General Entertainment,N/A,N/A,N/A,N/A,N/A,N/A


,Postal Code,1st Most Distinctive Venue,2nd Most Distinctive Venue,3rd Most Distinctive Venue,4th Most Distinctive Venue,5th Most Distinctive Venue,6th Most Distinctive Venue,7th Most Distinctive Venue,8th Most Distinctive Venue,9th Most Distinctive Venue,10th Most Distinctive Venue
0,M1B,Fast Food Restaurant,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1,M1C,Bar,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
2,M1E,Electronics Store,Moving Target,Rental Car Location,Mexican Restaurant,Breakfast Spot,Bank,Intersection,Medical Center,N/A,N/A
3,M1G,Coffee Shop,Mexican Restaurant,Korean Restaurant,Convenience Store,N/A,N/A,N/A,N/A,N/A,N/A
4,M1H,Hakka Restaurant,Fried Chicken Joint,Lounge,Bank,Athletics & Sports,Caribbean Restaurant,Thai Restaurant,Gas Station,Bakery,N/A
5,M1J,Pizza Place,Playground,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
6,M1K,Chinese Restaurant,Bus Station,Coffee Shop,Hobby Shop,Department Store,N/A,N/A,N/A,N/A,N/A
7,M1L,Bakery,Park,Ice Cream Shop,Soccer Field,Bus Station,Bus Line,N/A,N/A,N/A,N/A
8,M1M,American Restaurant,Motel,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
9,M1N,College Stadium,Skating Rink,Café,General Entertainment,N/A,N/A,N/A,N/A,N/A,N/A


#### Clustering into 5 clusters based on local venue normalized sums.

In [286]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Postal Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:kclusters] 

array([2, 2, 2, 2, 2], dtype=int32)

In [287]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Number of Venues', toronto_onehot.groupby('Postal Code').sum().reset_index(drop=True).sum(axis=1))
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
#neighborhoods_venues_sorted_distinct.insert(0, 'Distinctiveness', toronto_grouped.drop(columns='Postal Code').sum(axis=1))
neighborhoods_venues_sorted_distinct.insert(0, 'Number of Venues', toronto_onehot.groupby('Postal Code').sum().reset_index(drop=True).sum(axis=1))
neighborhoods_venues_sorted_distinct.insert(0, 'Cluster Labels', kmeans.labels_)

In [288]:
display(neighborhoods_venues_sorted.head())
neighborhoods_venues_sorted.shape

display(neighborhoods_venues_sorted_distinct.head())
neighborhoods_venues_sorted_distinct.shape

,Cluster Labels,Number of Venues,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,1.0,M1B,Fast Food Restaurant,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1,2,1.0,M1C,Bar,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
2,2,8.0,M1E,Electronics Store,Moving Target,Rental Car Location,Mexican Restaurant,Breakfast Spot,Bank,Intersection,Medical Center,N/A,N/A
3,2,5.0,M1G,Coffee Shop,Mexican Restaurant,Korean Restaurant,Convenience Store,N/A,N/A,N/A,N/A,N/A,N/A
4,2,9.0,M1H,Hakka Restaurant,Fried Chicken Joint,Lounge,Bank,Athletics & Sports,Caribbean Restaurant,Thai Restaurant,Gas Station,Bakery,N/A


,Cluster Labels,Number of Venues,Postal Code,1st Most Distinctive Venue,2nd Most Distinctive Venue,3rd Most Distinctive Venue,4th Most Distinctive Venue,5th Most Distinctive Venue,6th Most Distinctive Venue,7th Most Distinctive Venue,8th Most Distinctive Venue,9th Most Distinctive Venue,10th Most Distinctive Venue
0,2,1.0,M1B,Fast Food Restaurant,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1,2,1.0,M1C,Bar,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
2,2,8.0,M1E,Electronics Store,Moving Target,Rental Car Location,Mexican Restaurant,Breakfast Spot,Bank,Intersection,Medical Center,N/A,N/A
3,2,5.0,M1G,Coffee Shop,Mexican Restaurant,Korean Restaurant,Convenience Store,N/A,N/A,N/A,N/A,N/A,N/A
4,2,9.0,M1H,Hakka Restaurant,Fried Chicken Joint,Lounge,Bank,Athletics & Sports,Caribbean Restaurant,Thai Restaurant,Gas Station,Bakery,N/A


(103, 13)

In [299]:
toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Postal Code'), on='Postal Code')

toronto_merged # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Number of Venues,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2,3.0,Park,Food & Drink Shop,Bus Stop,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1,M4A,North York,Victoria Village,43.725882,-79.315572,2,4.0,Hockey Arena,Portuguese Restaurant,Coffee Shop,Pizza Place,N/A,N/A,N/A,N/A,N/A,N/A
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,45.0,Coffee Shop,Park,Pub,Bakery,Café,Breakfast Spot,Theater,Yoga Studio,Electronics Store,Ice Cream Shop
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,2,12.0,Clothing Store,Furniture / Home Store,Boutique,Miscellaneous Shop,Event Space,Accessories Store,Coffee Shop,Vietnamese Restaurant,N/A,N/A
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,35.0,Coffee Shop,Sushi Restaurant,Creperie,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burger Joint,Burrito Place,Café
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,2,1.0,Pizza Place,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,2,1.0,Fast Food Restaurant,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
7,M3B,North York,Don Mills,43.745906,-79.352188,2,5.0,Gym / Fitness Center,Café,Japanese Restaurant,Caribbean Restaurant,Baseball Field,N/A,N/A,N/A,N/A,N/A
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,2,11.0,Pizza Place,Gym / Fitness Center,Intersection,Athletics & Sports,Gastropub,Fast Food Restaurant,Bank,Pharmacy,Pet Store,Breakfast Spot
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,4,100.0,Clothing Store,Coffee Shop,Bubble Tea Shop,Middle Eastern Restaurant,Italian Restaurant,Café,Japanese Restaurant,Cosmetics Shop,Fast Food Restaurant,Ramen Restaurant


In [300]:
#toronto_merged[['Cluster Labels']] = toronto_merged[['Cluster Labels']].fillna(value=-1).astype('int32')

#### Mapping the 5 clusters on the city map

In [301]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
#adding seventh for Nan->6 category
#color6 = 'k';
#rainbow = rainbow + [color6]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postal Code'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [302]:
def freqtable(df):
    #res = pd.Series(df.set_index('Postal Code').drop(columns=['Number of Venues','Distinctiveness']).to_numpy().flatten()).value_counts()
    res = pd.Series(df.set_index('Postal Code').drop(columns=['Number of Venues']).to_numpy().flatten()).value_counts()
    res = res/df['Postal Code'].count()
    res_df = pd.DataFrame(res).T
    #res_df.insert(0,'Min Distinctiveness',df['Distinctiveness'].min())
    #res_df.insert(0,'Max Distinctiveness',df['Distinctiveness'].max())
    #res_df.insert(0,'Mean Distinctiveness',df['Distinctiveness'].mean())
    res_df.insert(0,'Min Number of Venues',df['Number of Venues'].min())
    res_df.insert(0,'Max Number of Venues',df['Number of Venues'].max())
    res_df.insert(0,'Mean Number of Venues',df['Number of Venues'].mean())
    return res_df

In [303]:
for i in range(0,kclusters):
    toronto_cluster1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == i, toronto_merged.columns[[0] + list(range(6, toronto_merged.shape[1]))]]
    print('Cluster {}:'.format(i))
    display(toronto_cluster1)
    display(freqtable(toronto_cluster1))

Cluster 0:


,Postal Code,Number of Venues,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,M5E,56.0,Coffee Shop,Cocktail Bar,Pub,Café,Restaurant,Beer Bar,Seafood Restaurant,Bakery,Cheese Shop,Farmers Market
23,M4G,32.0,Coffee Shop,Burger Joint,Bank,Furniture / Home Store,Sporting Goods Shop,Fish & Chips Shop,Sandwich Place,Supermarket,Dessert Shop,Sports Bar
25,M6G,16.0,Grocery Store,Café,Park,Candy Store,Diner,Italian Restaurant,Baby Store,Restaurant,Nightclub,Coffee Shop
37,M6J,44.0,Bar,Restaurant,Asian Restaurant,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Men's Store,Yoga Studio,Miscellaneous Shop,Brewery
41,M4K,42.0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Restaurant,Yoga Studio,Indian Restaurant,Dessert Shop,Pub
43,M6K,22.0,Café,Coffee Shop,Breakfast Spot,Grocery Store,Bakery,Convenience Store,Performing Arts Venue,Pet Store,Climbing Gym,Restaurant
54,M4M,40.0,Café,Coffee Shop,Gastropub,Brewery,American Restaurant,Bakery,Clothing Store,Latin American Restaurant,Stationery Store,Ice Cream Shop
55,M5M,24.0,Restaurant,Coffee Shop,Sushi Restaurant,Sandwich Place,Italian Restaurant,Thai Restaurant,Pharmacy,Butcher,Café,Pub
59,M2N,33.0,Ramen Restaurant,Café,Pizza Place,Restaurant,Sandwich Place,Coffee Shop,Vietnamese Restaurant,Sushi Restaurant,Discount Store,Bubble Tea Shop
74,M5R,21.0,Sandwich Place,Café,Coffee Shop,Park,History Museum,Shoe Repair,Burger Joint,Indian Restaurant,Middle Eastern Restaurant,BBQ Joint


,Mean Number of Venues,Max Number of Venues,Min Number of Venues,Coffee Shop,Café,Restaurant,Italian Restaurant,Bakery,Sandwich Place,Pub,Pizza Place,Dessert Shop,Bar,Sushi Restaurant,Yoga Studio,Grocery Store,Brewery,Indian Restaurant,Pharmacy,Burger Joint,Diner,Butcher,Bank,Ice Cream Shop,Park,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Furniture / Home Store,Candy Store,Fish & Chips Shop,Middle Eastern Restaurant,Latin American Restaurant,Ramen Restaurant,Nightclub,Supermarket,Greek Restaurant,Sports Bar,Farmers Market,Pet Store,Shoe Repair,Gym,Convenience Store,Breakfast Spot,French Restaurant,Asian Restaurant,Mexican Restaurant,Bubble Tea Shop,Tea Room,History Museum,Comfort Food Restaurant,Miscellaneous Shop,Clothing Store,Beer Bar,Stationery Store,Baby Store,Liquor Store,BBQ Joint,Thai Restaurant,Cocktail Bar,American Restaurant,Bookstore,Seafood Restaurant,Japanese Restaurant,Gastropub,Discount Store,Performing Arts Venue,Sporting Goods Shop,Gaming Cafe,Climbing Gym,Men's Store,Cheese Shop
0,36.066667,58.0,16.0,0.933333,0.866667,0.6,0.466667,0.4,0.333333,0.333333,0.266667,0.266667,0.266667,0.266667,0.2,0.2,0.2,0.2,0.133333,0.133333,0.133333,0.133333,0.133333,0.133333,0.133333,0.133333,0.133333,0.133333,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667


Cluster 1:


,Postal Code,Number of Venues,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,45.0,Coffee Shop,Park,Pub,Bakery,Café,Breakfast Spot,Theater,Yoga Studio,Electronics Store,Ice Cream Shop
4,M7A,35.0,Coffee Shop,Sushi Restaurant,Creperie,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burger Joint,Burrito Place,Café
15,M5C,79.0,Coffee Shop,Café,Cocktail Bar,Gastropub,American Restaurant,Restaurant,Beer Bar,Gym,Italian Restaurant,Lingerie Store
24,M5G,63.0,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Salad Place,Bubble Tea Shop,Burger Joint,Japanese Restaurant,Department Store,Restaurant
99,M4Y,79.0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Yoga Studio,Bubble Tea Shop,Pub,Burger Joint,Café


...

,Postal Code,Number of Venues,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,M5H,93.0,Coffee Shop,Café,Restaurant,Clothing Store,Gym,Deli / Bodega,Thai Restaurant,Hotel,Bakery,American Restaurant
36,M5J,100.0,Coffee Shop,Aquarium,Hotel,Café,Fried Chicken Joint,Sporting Goods Shop,Scenic Lookout,Italian Restaurant,Restaurant,Brewery
42,M5K,100.0,Coffee Shop,Hotel,Café,Restaurant,Italian Restaurant,American Restaurant,Seafood Restaurant,Deli / Bodega,Salad Place,Japanese Restaurant
48,M5L,100.0,Coffee Shop,Café,Restaurant,Hotel,Gym,American Restaurant,Seafood Restaurant,Japanese Restaurant,Italian Restaurant,Deli / Bodega
92,M5W,93.0,Coffee Shop,Café,Japanese Restaurant,Italian Restaurant,Pub,Restaurant,Cocktail Bar,Seafood Restaurant,Beer Bar,Breakfast Spot
97,M5X,100.0,Coffee Shop,Café,Hotel,Restaurant,Gym,Japanese Restaurant,Seafood Restaurant,Salad Place,Steakhouse,Deli / Bodega


,Mean Number of Venues,Max Number of Venues,Min Number of Venues,Café,Coffee Shop,Restaurant,Hotel,Japanese Restaurant,Deli / Bodega,Seafood Restaurant,Italian Restaurant,Gym,American Restaurant,Salad Place,Thai Restaurant,Sporting Goods Shop,Clothing Store,Cocktail Bar,Pub,Bakery,Fried Chicken Joint,Steakhouse,Aquarium,Breakfast Spot,Brewery,Scenic Lookout,Beer Bar
0,97.666667,100.0,93.0,1.0,1.0,1.0,0.833333,0.666667,0.666667,0.666667,0.666667,0.5,0.5,0.333333,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667


Cluster 4:


,Postal Code,Number of Venues,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,M5B,100.0,Clothing Store,Coffee Shop,Bubble Tea Shop,Middle Eastern Restaurant,Italian Restaurant,Café,Japanese Restaurant,Cosmetics Shop,Fast Food Restaurant,Ramen Restaurant
33,M2J,67.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Japanese Restaurant,Bank,Convenience Store,Food Court,Shoe Store,Women's Store


,Mean Number of Venues,Max Number of Venues,Min Number of Venues,Clothing Store,Coffee Shop,Japanese Restaurant,Fast Food Restaurant,Ramen Restaurant,Middle Eastern Restaurant,Café,Cosmetics Shop,Bank,Food Court,Restaurant,Women's Store,Convenience Store,Italian Restaurant,Bubble Tea Shop,Shoe Store
0,83.5,100.0,67.0,1.0,1.0,1.0,1.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5


#### Cluster 0 seems to have a moderate average number of venues with Coffee Shop and then Cafes being most prevalent

#### Cluster 1 seems to have a higher average number of venues with Coffee Shop and Cafes still being most prevalent.

#### Cluster 2 seems to have the lowest venue density areas with the lowest average number of venues, and having varied categories.

#### Cluster 3 has the highest average number of venues again with Cafes and Coffee Shops as most prevalent, but also with a high frequency of hotels and ethnic restaurants.

#### Cluster 4 has high venue density but with Clothing Stores as the most prevalent.